In [8]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class Maxout(nn.Module):
    def __init__(self, input_dim, output_dim, num_pieces):
        super(Maxout, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_pieces = num_pieces
        self.fc = nn.Linear(input_dim, output_dim * num_pieces)

    def forward(self, x):
        output = self.fc(x)
        output = output.view(-1, self.output_dim, self.num_pieces)
        output, _ = output.max(dim=2)
        return output

class MaxoutNetwork(nn.Module):
    def __init__(self):
        super(MaxoutNetwork, self).__init__()
        self.fc1 = Maxout(784, 256, 2)  # 2 pieces for each Maxout unit
        self.fc2 = Maxout(256, 10, 2)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.fc1(x)
        x = self.fc2(x)
        return x
    
class FFN(nn.Module):
    def __init__(self):
        super(FFN, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 10)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


import torch.optim as optim
from torchvision import datasets, transforms

# Load the MNIST dataset
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

trainset = datasets.MNIST(root='./data/MNIST', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.MNIST(root='./data/MNIST', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Initialize the model, loss function, and optimizer
models = [MaxoutNetwork(), FFN()]
criterion = nn.CrossEntropyLoss()
optimizers = [optim.SGD(model.parameters(), lr=0.01, momentum=0.9) for model in models]

for i, model in enumerate(models):
    # Train the model
    optimizer = optimizers[i]
    for epoch in range(1):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            # if i % 2000 == 1999:    # print every 2000 mini-batches
            #     print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            #     running_loss = 0.0

    print('Finished Training')

    # Test the model
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Finished Training
Accuracy of the network on the 10000 test images: 96 %
Finished Training
Accuracy of the network on the 10000 test images: 92 %
